In [1]:
import numpy as np
from helpers.utils import print_metrics, executeIrisOneVsRest
from helpers.icq_methods import create_and_execute_classifier_new_approach

In [2]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# We're ignoring some warning from sklearn.metrics.classification_report
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)

In [3]:
import sys
import os
sys.path.append(os.path.abspath('models/'))

In [4]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from helpers.database_helpers import get_stratified_kfold, get_iris
from models.icq_estimators import IcqClassifier
from helpers.icq_methods import create_and_execute_classifier_new_approach

In [5]:
seed = 42

In [6]:
X, y = get_iris()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
cv = get_stratified_kfold(random_seed=seed)
icq = OneVsRestClassifier(IcqClassifier(classifier_function=create_and_execute_classifier_new_approach, max_iter=2000, random_seed=seed, plot_graphs_and_metrics=False))

In [5]:
%%time
scores = []
f1scores = []
for i_random_state in range(10, 70, 5):
    curr_scores, curr_f1scores = executeIrisOneVsRest(random_seed=i_random_state,
                                                classifier_function=create_and_execute_classifier_new_approach,
                                                sigma_q_weights=[0, 11, 9, 2],
                                                max_iter=2000,
                                                print_each_fold_metric=False,
                                                print_avg_metric=True)
    scores.append(np.mean(curr_scores))
    f1scores.append(np.mean(curr_f1scores))

AVG: Scores = 0.33333333333333337 F1-Scores = 0.17222222222222222
AVG: Scores = 0.38 F1-Scores = 0.22453703703703698
AVG: Scores = 0.49333333333333335 F1-Scores = 0.37093967407434897
AVG: Scores = 0.3 F1-Scores = 0.1658554064900814
AVG: Scores = 0.3 F1-Scores = 0.15
AVG: Scores = 0.6333333333333333 F1-Scores = 0.5289729225023342
AVG: Scores = 0.41999999999999993 F1-Scores = 0.2801585253984635
AVG: Scores = 0.33333333333333337 F1-Scores = 0.16666666666666669
AVG: Scores = 0.36000000000000004 F1-Scores = 0.20820105820105822
AVG: Scores = 0.33333333333333337 F1-Scores = 0.16666666666666669
AVG: Scores = 0.62 F1-Scores = 0.5064814814814814
AVG: Scores = 0.6333333333333333 F1-Scores = 0.5166666666666666
Wall time: 4h 13min 42s


In [8]:
params = dict()
params["estimator__sigma_q_weights"] = []
params["estimator__classifier_function"] = [create_and_execute_classifier_new_approach]
params["estimator__max_iter"] = [2000]
params["estimator__random_seed"] = [seed]
params["estimator__accuracy_succ"] = [1.00]
for i in range(0, 5, 1):
    params["estimator__sigma_q_weights"].append([i, i, i, i])

In [9]:
%%time
busca = RandomizedSearchCV(icq, params, n_iter=10, scoring='accuracy', n_jobs=-1, cv=cv, random_state=seed)

# Executa busca
resultado = busca.fit(X_train, y_train)

# Resume resultados
print('Resultados busca - ICQ Alterado')
print('Melhor acurácia: %s' % resultado.best_score_)
print('Melhor hiperparâmetro: %s' % resultado.best_params_)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Resultados busca - ICQ Alterado
Melhor acurácia: 0.6416666666666667
Melhor hiperparâmetro: {'estimator__sigma_q_weights': [2, 2, 2, 2], 'estimator__random_seed': 42, 'estimator__max_iter': 2000, 'estimator__classifier_function': <function create_and_execute_classifier_new_approach at 0x00000280A6918DC0>, 'estimator__accuracy_succ': 1.0}
Wall time: 43min 35s


In [10]:
params = dict()
params["estimator__sigma_q_weights"] = [[1,1,1,0]]
params["estimator__classifier_function"] = [create_and_execute_classifier_new_approach]
params["estimator__max_iter"] = [2000]
params["estimator__random_seed"] = [seed]
params["estimator__accuracy_succ"] = [1.0]
params["estimator__learning_rate"] = [0.1, 0.01, 0.001, 0.2, 0.02, 0.002, 0.0001]

In [11]:
%%time
busca = RandomizedSearchCV(icq, params, n_iter=10, scoring='accuracy', n_jobs=-1, cv=cv, random_state=seed)

# Executa busca
resultado = busca.fit(X_train, y_train)

# Resume resultados
print('Resultados busca - ICQ Alterado')
print('Melhor acurácia: %s' % resultado.best_score_)
print('Melhor hiperparâmetro: %s' % resultado.best_params_)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 7 is smaller than n_iter=10. Running 7 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Resultados busca - ICQ Alterado
Melhor acurácia: 0.9583333333333333
Melhor hiperparâmetro: {'estimator__sigma_q_weights': [1, 1, 1, 0], 'estimator__random_seed': 42, 'estimator__max_iter': 2000, 'estimator__learning_rate': 0.001, 'estimator__classifier_function': <function create_and_execute_classifier_new_approach at 0x00000280A6918DC0>, 'estimator__accuracy_succ': 1.0}
Wall time: 46min 24s


In [5]:
%%time
scores = []
f1scores = []
for i_random_state in range(10, 70, 5):
    curr_scores, curr_f1scores = executeIrisOneVsRest(random_seed=i_random_state,
                                                classifier_function=create_and_execute_classifier_new_approach,
                                                sigma_q_weights=[1, 1, 1, 0],
                                                max_iter=2000,
                                                print_each_fold_metric=False,
                                                print_avg_metric=True,
                                                learning_rate=0.001)
    scores.append(np.mean(curr_scores))
    f1scores.append(np.mean(curr_f1scores))

AVG: Scores = 0.9066666666666666 F1-Scores = 0.9056228956228957
AVG: Scores = 0.9533333333333334 F1-Scores = 0.9524410774410775
AVG: Scores = 0.9400000000000001 F1-Scores = 0.9386868686868686
AVG: Scores = 0.9200000000000002 F1-Scores = 0.919040404040404
AVG: Scores = 0.8866666666666667 F1-Scores = 0.8801394901394902
AVG: Scores = 0.9400000000000001 F1-Scores = 0.9391750841750841
AVG: Scores = 0.9533333333333334 F1-Scores = 0.9524410774410773
AVG: Scores = 0.9400000000000001 F1-Scores = 0.9382491582491582
AVG: Scores = 0.96 F1-Scores = 0.9588720538720539
AVG: Scores = 0.9533333333333334 F1-Scores = 0.9533164983164983
AVG: Scores = 0.9333333333333333 F1-Scores = 0.9305977355977355
AVG: Scores = 0.9466666666666667 F1-Scores = 0.9463299663299664
Wall time: 4h 1min 10s


In [6]:
print_metrics(scores, f1scores)

Scores: [0.9066666666666666, 0.9533333333333334, 0.9400000000000001, 0.9200000000000002, 0.8866666666666667, 0.9400000000000001, 0.9533333333333334, 0.9400000000000001, 0.96, 0.9533333333333334, 0.9333333333333333, 0.9466666666666667]
Best score: 0.96
F1-Scores: [0.9056228956228957, 0.9524410774410775, 0.9386868686868686, 0.919040404040404, 0.8801394901394902, 0.9391750841750841, 0.9524410774410773, 0.9382491582491582, 0.9588720538720539, 0.9533164983164983, 0.9305977355977355, 0.9463299663299664]
Max F1-Score: 0.9588720538720539


In [7]:
print("Avg score:", np.mean(scores))
print("Avg F1-Score:", np.mean(f1scores))

Avg score: 0.9361111111111112
Avg F1-Score: 0.9345760258260257


In [7]:
params = dict()
params["estimator__sigma_q_weights"] = [[1,1,1,0]]
params["estimator__classifier_function"] = [create_and_execute_classifier_new_approach]
params["estimator__max_iter"] = [2000]
params["estimator__random_seed"] = [seed]
params["estimator__accuracy_succ"] = [1.0]
params["estimator__learning_rate"] = []
for i in range (1, 10):
    params["estimator__learning_rate"].append(0.1 * i)
    params["estimator__learning_rate"].append(0.01 * i)
    params["estimator__learning_rate"].append(0.001 * i)
    params["estimator__learning_rate"].append(0.0001 * i)
    params["estimator__learning_rate"].append(0.00001 * i)
    params["estimator__learning_rate"].append(0.000001 * i)

In [8]:
%%time
busca = GridSearchCV(icq, params, scoring='accuracy', n_jobs=-1, cv=cv, verbose=10)

# Executa busca
resultado = busca.fit(X_train, y_train)

# Resume resultados
print('Resultados busca - ICQ Alterado')
print('Melhor acurácia: %s' % resultado.best_score_)
print('Melhor hiperparâmetro: %s' % resultado.best_params_)

Fitting 10 folds for each of 54 candidates, totalling 540 fits
Resultados busca - ICQ Alterado
Melhor acurácia: 0.9666666666666666
Melhor hiperparâmetro: {'estimator__accuracy_succ': 1.0, 'estimator__classifier_function': <function create_and_execute_classifier_new_approach at 0x000001F85E6CE8B0>, 'estimator__learning_rate': 0.0008, 'estimator__max_iter': 2000, 'estimator__random_seed': 42, 'estimator__sigma_q_weights': [1, 1, 1, 0]}
Wall time: 8h 24min 59s


In [10]:
%%time
scores = []
f1scores = []
for i_random_state in range(10, 70, 5):
    curr_scores, curr_f1scores = executeIrisOneVsRest(random_seed=i_random_state,
                                                classifier_function=create_and_execute_classifier_new_approach,
                                                sigma_q_weights=[1, 1, 1, 0],
                                                max_iter=2000,
                                                print_each_fold_metric=False,
                                                print_avg_metric=True,
                                                learning_rate=0.0008)
    scores.append(np.mean(curr_scores))
    f1scores.append(np.mean(curr_f1scores))

AVG: Scores = 0.9200000000000002 F1-Scores = 0.9188047138047137
AVG: Scores = 0.9533333333333334 F1-Scores = 0.9533346283346283
AVG: Scores = 0.9533333333333334 F1-Scores = 0.9527104377104377
AVG: Scores = 0.9266666666666665 F1-Scores = 0.9238637288637289
AVG: Scores = 0.8866666666666667 F1-Scores = 0.8805603655603657
AVG: Scores = 0.9466666666666667 F1-Scores = 0.9458417508417508
AVG: Scores = 0.9466666666666667 F1-Scores = 0.9457070707070706
AVG: Scores = 0.9400000000000001 F1-Scores = 0.9391750841750841
AVG: Scores = 0.9533333333333334 F1-Scores = 0.9521380471380472
AVG: Scores = 0.9400000000000001 F1-Scores = 0.9397811447811447
AVG: Scores = 0.9 F1-Scores = 0.8972138972138971
AVG: Scores = 0.9466666666666667 F1-Scores = 0.9461952861952861
Wall time: 5h 37min 51s


In [11]:
print_metrics(scores, f1scores)

Scores: [0.9200000000000002, 0.9533333333333334, 0.9533333333333334, 0.9266666666666665, 0.8866666666666667, 0.9466666666666667, 0.9466666666666667, 0.9400000000000001, 0.9533333333333334, 0.9400000000000001, 0.9, 0.9466666666666667]
Best score: 0.9533333333333334
F1-Scores: [0.9188047138047137, 0.9533346283346283, 0.9527104377104377, 0.9238637288637289, 0.8805603655603657, 0.9458417508417508, 0.9457070707070706, 0.9391750841750841, 0.9521380471380472, 0.9397811447811447, 0.8972138972138971, 0.9461952861952861]
Max F1-Score: 0.9533346283346283


In [12]:
print("Avg score:", np.mean(scores))
print("Avg F1-Score:", np.mean(f1scores))

Avg score: 0.9344444444444445
Avg F1-Score: 0.9329438462771796
